In [1]:
import numpy as np
import sys
import os
import pandas as pd
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))# to import rootID_lookup and authentication_utils like below
import rootID_lookup as IDlook
import authentication_utils as auth

In [2]:
# auth = client.auth
# auth.get_new_token()
# new_token = '' # This is the text you see after you visit the website.
# auth.save_token(token=new_token)

In [7]:
# check datastack_name here https://globalv1.daf-apis.com/info/
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.info.get_datastack_info()

{'synapse_table': None,
 'soma_table': None,
 'segmentation_source': 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod',
 'description': 'This is the first version of the fanc dataset hosted in CAVE.',
 'analysis_database': None,
 'aligned_volume': {'image_source': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/fanc_v4_em',
  'id': 7,
  'name': 'fanc_v4',
  'description': 'This is the 4th alignment of the female adult nerve cord dataset.'},
 'local_server': 'https://cave.fanc-fly.com',
 'viewer_site': None}

In [8]:
client.annotation.get_tables()

https://cave.fanc-fly.com/annotation/api/v2/aligned_volume/fanc_v4/table


['t1mn', 'test_soma', 'synapses_may2021']

In [17]:
df = pd.read_csv('../Output/body_info_Aug2021.csv', header=0)

In [ ]:
df = df.reindex(columns=['nuc_xyz', 'nuc_svID', 'nuc_rootID', 'body_xyz', 'body_svID', 'body_rootID'])
df = df.rename(columns={'body_svID': 'soma_svID', 'body_rootID': 'soma_rootID'})

In [18]:
df

,nuc_xyz,nuc_svID,nuc_rootID,body_xyz,soma_svID,soma_rootID
0,"(64344, 155528, 2733)",74244180477808300,0,"(64096, 155520, 2735)",74244180477821787,0
1,"(17088, 157440, 2646)",72625768014147752,0,"(16800, 157360, 2680)",72625768014223466,0
2,"(40448, 196000, 1217)",73401129131948686,0,"(40144, 195856, 1213)",73401129131935868,0
3,"(52664, 154184, 2631)",73821967945765967,0,"(52320, 153968, 2663)",73821967945813511,0
4,"(27208, 86824, 2002)",72975274937293138,0,"(26928, 86432, 1996)",72975274937264610,0
...,...,...,...,...,...,...
17071,"(61208, 110496, 1436)",74101930422511045,0,"(59824, 110608, 1362)",74101999142110560,0
17072,"(49280, 117624, 915)",73679992567191019,0,"(47616, 117376, 885)",73679992567212870,0
17073,"(14352, 124520, 3228)",72554300093870036,0,"(13184, 124416, 3238)",72483931349546701,0
17074,"(9072, 128072, 2373)",72272962019259746,0,"(8032, 127680, 2353)",72272962086203920,0


In [21]:
example_schema = client.schema.schema_definition('nucleus_detection')
example_schema

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'definitions': {'SpatialPoint': {'required': ['position'],
   'type': 'object',
   'properties': {'position': {'type': 'array'}},
   'additionalProperties': False},
  'BoundSpatialPoint': {'required': ['position'],
   'type': 'object',
   'properties': {'position': {'type': 'array'},
    'root_id': {'type': 'integer'},
    'supervoxel_id': {'type': 'integer'}},
   'additionalProperties': False},
  'NucleusDetection': {'required': ['pt'],
   'type': 'object',
   'properties': {'bb_end': {'type': 'object',
     '$ref': '#/definitions/SpatialPoint',
     'description': 'high corner of the bounding box'},
    'bb_start': {'type': 'object',
     '$ref': '#/definitions/SpatialPoint',
     'description': 'low corner of the bounding box'},
    'pt': {'type': 'object',
     '$ref': '#/definitions/BoundSpatialPoint',
     'description': 'the point to be used for attaching objects to the dynamic segmentation'},
    'valid': {'title': 'valid'

In [35]:
client.annotation.create_table(table_name='nucleus_aug2021',
                               schema_name='nucleus_detection',
                               description = 'xxx',
                               voxel_resolution =[4.3, 4.3, 45.0])

<Response [200]>

In [94]:
df_test = pd.DataFrame([{'id':0,
         'type': 'nucleus_detection',
         'pt_position': [40448, 196000, 1217],
         'supervoxel_id': 73401129131948686},
        {'id':1,
        'type': 'nucleus_detection',
        'pt_position': [17088, 157440, 2646],
        'supervoxel_id': 72625768014147752}])

In [95]:
df_test

,id,type,pt_position,supervoxel_id
0,0,nucleus_detection,"[40448, 196000, 1217]",73401129131948686
1,1,nucleus_detection,"[17088, 157440, 2646]",72625768014147752


In [103]:
client.annotation.post_annotation_df('test_soma', df_test, position_columns=['pt_position'])

'Inserted 2 annotations'

In [104]:
client.annotation.get_annotation_count('test_soma')

2

In [105]:
client.annotation.get_annotation('test_soma',annotation_ids=[1])

HTTPError: 502 Server Error: Bad Gateway for url: https://cave.fanc-fly.com/annotation/api/v2/aligned_volume/fanc_v4/table/test_soma/annotations?annotation_ids=1 content:b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body>\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.17.10</center>\r\n</body>\r\n</html>\r\n'